In [1]:
import sys

sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt

from VQE import VQE
from utils import number2SummedOp

from joblib import Parallel, delayed

from qiskit import IBMQ
from qiskit import Aer
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.utils.quantum_instance import QuantumInstance
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA, COBYLA
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.circuit.library import EfficientSU2

In [2]:
def callback(evals, params, mean, deviation):
    stream = getattr(sys, "stdout")
    #print("{}, {}".format(evals, mean), file=stream)
    stream.flush()

In [3]:
def run_VQE(solver, qubitOp, seed, nmax=100):
    np.random.seed(seed)

    solution = False
    while not solution and nmax > 0:
        try:
            solver.compute_minimum_eigenvalue(qubitOp)
            solution = True
        except Exception:
            print('Trying again...')
            nmax -= 1

    if not solution:
        return None

    return solver.energies

In [4]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic')
name_backend = 'ibmq_montreal'
backend_device = provider.get_backend(name_backend)
backend_sim = Aer.get_backend('aer_simulator')
coupling_map = backend_device.configuration().coupling_map

In [5]:
qi = QuantumInstance(backend=backend_device,shots=2**13)

In [6]:
#Molecules = [ 'H20', 'BeH2', 'LiH' ]
Molecules = ['BeH2']

In [7]:
for molecule in Molecules:
    
    data = np.load('../data/optimal_grouping_' + molecule + '_' + name_backend + '.npy', allow_pickle=True).item()
    qubit_op = number2SummedOp(data['optimal_labels'], data['optimal_coeffs'])

    init_state = data['init_state']

    num_qubits = qubit_op.num_qubits
    ansatz = init_state.compose(EfficientSU2(num_qubits, ['ry', 'rz'], entanglement='linear', reps=1))
    num_var = ansatz.num_parameters
    initial_params = [0.1] * num_var

    maxiter = 200
    optimizer = SPSA(maxiter=maxiter, last_avg=1)

    solvers_TPB  = VQE(ansatz, optimizer, initial_params, grouping='TPB',
                        quantum_instance=qi, callback=callback) 

    solvers_EM   = VQE(ansatz, optimizer, initial_params, grouping='Entangled',
                        quantum_instance=qi, callback=callback) 

    solvers_HEEM = VQE(ansatz, optimizer, initial_params, grouping='Entangled',
                        quantum_instance=qi, connectivity=coupling_map, callback=callback) 

    results_TPB  = run_VQE(solvers_TPB, qubit_op, None)
    
    results_EM   = run_VQE(solvers_EM, qubit_op, None)
    
    results_HEEM = run_VQE(solvers_HEEM, qubit_op, None)
    
    np.save( 'results_'+molecule, np.array( [results_TPB, 
                                              results_EM,
                                              results_HEEM] ) )
    

C:\Users\lucia\miniconda3\lib\site-packages\qiskit\utils\run_circuits.py:695: UserWarning: max_credits is not a recognized runtime option and may be ignored by the backend.
  return backend.run(circuits, **run_kwargs)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\lucia\miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lucia\AppData\Local\Temp/ipykernel_19592/1699615574.py", line 25, in <module>
    results_TPB  = run_VQE(solvers_TPB, qubit_op, None)
  File "C:\Users\lucia\AppData\Local\Temp/ipykernel_19592/2526814013.py", line 7, in run_VQE
    solver.compute_minimum_eigenvalue(qubitOp)
  File "C:\Users\lucia\Nextcloud\Programas\Github\Hardware-efficient-variational-quantum-eigensolver-with-entangled-measurements\Codes\experiments\..\VQE.py", line 375, in compute_minimum_eigenvalue
    vqresult = self.find_minimum(initial_point=self._initial_point,
  File "C:\Users\lucia\Nextcloud\Programas\Github\Hardware-efficient-variational-quantum-eigensolver-with-entangled-measurements\Codes\experiments\..\VQE.py", line 482, in find_minimum
    opt_params, opt_val, num_optimizer_evals = optimizer.o

TypeError: object of type 'NoneType' has no len()

In [ ]:
plt.plot( results_TPB, label='TPB' )
plt.plot( results_EM, label='EM' )
plt.plot( results_HEEM, label='HEEM' )
plt.legend()